In [37]:
import requests
import base64
from pprint import pprint

def get_data(namespace_id:str, height:int):
    r = requests.get(f'http://localhost:26658/namespaced_data/{namespace_id}/height/{height}')
    return dict(r.json())

In [ ]:
def parse(data: str):
    decoded_data = base64.b64decode(data).decode('utf-8')
    return decoded_data

In [38]:
def post_data(namespace_id, data, gas_limit=70000):
    data = data.encode('utf-8')
    d = data.hex()
    r = requests.post(f'http://localhost:26658/submit_pfd', json={"namespace_id":namespace_id,"data":d,"gas_limit":gas_limit})
    pprint(r.json()) 
    print(get_data('756f60cbe7bf5401',r.json()['height']))

In [ ]:
post_data('756f60cbe7bf5401', "{'Pratham': 'prasoon.cel'}")

In [ ]:
# {"type": "REGISTER_NAME", "payload": {"name": "Pratham"}, signature: "123"}
state = dict() # initial state is empty

# TODO: real implementation
def get_address(signature):
    # pretend like the address is the signature
    return signature

# can be improved
def is_valid_address(address):
    return address == str

def update_state(action):
    actionType = action.get("type")
    address = get_address(action.get("signature"))
    if actionType is "REGISTER_NAME":
        name = action.get("payload").get("name")
        state.setdefault(name, address)
    elif actionType is "TRANSFER_NAME":
        name = action.get("payload").get("name")
        recipient = action.get("payload").get("recipient")
        if state.get(name) is address and is_valid_address(recipient):
            state.set(name, recipient)

#

In [45]:
x = dict(dict('str' : 123) : 5)

TypeError: dict expected at most 1 argument, got 2

# Historical Syncing
We need to start from block 1 (or whatever block was the latest at the time that we launch this protocol) and process each block until the we get to the latest block.

In [ ]:
my_namespace = '756f60cbe7bf5401'
initialHeight = 49350
latestHeight = 49432 # for example
for height in range(initialHeight, latestHeight):
    data_list = get_data(my_namespace, height)
    try:
        parsed_data = parse(data_list.get('data')[0])
        update_state(dict(eval(parsed_data)))
    except Exception:
        pass